In [1]:
import copy
import random
import pandas as pd
import numpy as np
import time
import random
from IPython.display import clear_output

# Battleship

Rules:
1. Each player arranges ships according to fleet
2. Take turns firing a shot
3. Mark Hits and Misses
4. Call out when a ship has been sunk
5. Sink all to win

Ships:
1. Carrier - 5
2. Battleship - 4
3. Cruiser - 3
4. Submarine - 3
5. Destroyer -2

10 Rows x 10 Columns  

|   |   |   |   |   |   |   |   |   |   |
|---|---|---|---|---|---|---|---|---|---|
|   |   |   |   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |
|   |   |   |   |   |   |   |   |   |   |

##### Parameters

In [2]:
Enemy_Ships = [["Carrier", 5, []],
               ["Battleship", 4, []],
               ["Cruiser", 3, []],
               ["Submarine", 3, []],
               ["Destroyer", 2, []]]

##### Helper Functions

In [3]:
# Is current proposed ship allocation valid?

def ValidPlacement(Board, i, j, k, length):
    # Board = Dictionary for Board
    # i = row of ship
    # j = column of ship
    # k = orientation of ship (0 = hoz, 1 = vert)
    # length = length of ship
    
    #Horizontal Ship
    if (k == 0):
        if (j>11-length):
            return 0
        else:
            occupied = 0
            for l in range(length):
                occupied = occupied + Board[i][j+l]
            if(occupied == 0):
                return 1
            else:
                return 0
            
    #Vertical Ship
    if (k == 1):
        if (i>11-length):
            return 0
        else:
            occupied = 0
            for l in range(length):
                occupied = occupied + Board[i+l][j]
            if(occupied == 0):
                return 1
            else:
                return 0

In [4]:
def ValidLocations(Board, Ship_List):
    #Wri
    #Board: dictionary for occupied spaces
    #Ship_List: Array with ship name, length, 3rd index to place valid locations 
    #          ["Ship_Name", length, []]
    
    for s in range(len(Ship_List)):
        valid_placements = []
        Ship = Ship_List[s]
        length = Ship[1]

        for i in range(1,11):
            for j in range(1,11):
                for k in range(2):
                     if (ValidPlacement(Board,i,j,k,length)==1):
                            valid_placements.append([i,j,k])
        Ship[2] = valid_placements

In [5]:
def SpacesOccupied(length,location):
    #length = length of ship
    #location = [i,j,k]
    #Returns list of spaces occupied by ship
    
    occupied = []
    i,j,k = location
    
    if (k==0):
        for l in range(length):
            occupied.append((i,j+l))
    if (k==1):
        for l in range(length):
            occupied.append((i+l,j))
        
    return occupied

In [6]:
def RandomSample(Ship_List):
    # Ship_List = ["Ship_Name", length, locations]
    # Return: Data frame of board with 1s for ship positions
    #        or all 0s if not a valid arrangement
    
    PDF = pd.DataFrame(index=range(1,11), columns=range(1,11))
    PDF = PDF.fillna(0) # with 0s rather than NaNs
    
    locations = []
    for s in range(len(Ship_List)):
        Ship = Ship_List[s]
        position = random.choice(Ship[2])
        spaces = SpacesOccupied(Ship[1],position)
        locations.extend(spaces)
    #Check if valid configuration
    if(len(locations) == len(set(locations))):
        for t in locations:
            PDF.loc[t] = 1
        return PDF
    else:
        return False        

In [7]:
def NSamples(N,M,Ship_List):
    #N = Number of samples wanted
    #M = Max number of iterations
    #Ship_List = ["Ship_Name", length, locations]
    #Returns: (DF, S) Data frame of accumulated possible locations and number of actual samples taken
    
    PDF = pd.DataFrame(index=range(1,11), columns=range(1,11))
    PDF = PDF.fillna(0) # with 0s rather than NaNs
    n = 0
    m = 0
    while( (n<N) & (m<M) ):
        m = m + 1
        Result = RandomSample(Ship_List)
        if (type(Result) != bool):
            PDF = PDF + Result
            n = n+1
    
    return(PDF,n)

##### Initialize Board

In [8]:
def initBoard():
    Board = {}
    for i in range(1,11):
        Board[i] = {}
        for j in range(1,11):
            Board[i][j] = 0 
            
    return Board

In [9]:
Board = initBoard()

##### Example

In [10]:
Board[5][5] = 1
Board[5][6] = 1
Board[6][5] = 1
Board[6][6] = 1

Board[2][2] = 1
Board[3][3] = 1
Board[4][4] = 1
Board[5][1] = 1

Board[10][10] = 1
Board[8][3] = 1
Board[2][7] = 1
Board[10][9] = 1

In [11]:
def dispBoard(Board):
    #Board: dictionary for occupied spaces

    DispBoard = np.zeros((10,10))

    for i in range(1,11):
        for j in range(1,11):
            if Board[i][j] == 0:
                DispBoard[i-1][j-1] = 0
            else:
                DispBoard[i-1][j-1] = 1

    print(DispBoard)

In [12]:
ValidLocations(Board, Enemy_Ships)

In [13]:
PDF,n = NSamples(2000,6000,Enemy_Ships)

In [14]:
# PDF

In [15]:
PDF/n

,1,2,3,4,5,6,7,8,9,10
1,0.1135,0.1320,0.1870,0.2290,0.2550,0.2555,0.1995,0.2255,0.1825,0.1475
2,0.0845,0.0000,0.0585,0.1215,0.1515,0.1080,0.0000,0.1250,0.1350,0.1645
3,0.0950,0.0725,0.0000,0.0930,0.1920,0.2195,0.2205,0.2720,0.2395,0.2325
4,0.0845,0.1715,0.0775,0.0000,0.1040,0.1545,0.2560,0.2955,0.2735,0.2535
5,0.0000,0.2065,0.1100,0.0920,0.0000,0.0000,0.1870,0.2320,0.2370,0.2505
6,0.1105,0.2630,0.1345,0.1680,0.0000,0.0000,0.2175,0.2325,0.2365,0.2330
7,0.1605,0.2900,0.1890,0.3260,0.2290,0.2190,0.3320,0.3090,0.2560,0.2235
8,0.1415,0.1610,0.0000,0.2000,0.1780,0.2015,0.2975,0.2865,0.2095,0.1715
9,0.1700,0.2175,0.1725,0.2840,0.2695,0.2520,0.2640,0.2540,0.1565,0.1195
10,0.1150,0.1665,0.1805,0.2440,0.2210,0.1780,0.1500,0.1115,0.0000,0.0000


In [16]:
# n

In [17]:
# a = np.array([[1,3,2],[8,7,9],[5,6,4]])
# #print(a)
# arr = np.where(a==a.max())
# x = arr[0]
# y = arr[1]
# #print(a[x].flatten()[y])

In [18]:
def isHit(Enemies, x, y):
    # isHit = [Enemies, x, y]
    # Enemies: shadow board containing all occupied spaces
    # x, y: target x-y coordinates on board
    # Returns: whether or not it's a hit!
    
    if Enemies[x,y]:
        print('hit!')
        return 1
    else:
        print('miss!')
        return 0

In [19]:
def ShipSearch(Board, Enemy_Ships):
    # ShipSearch = [Board, Enemy_Ships]
    # Board: dictionary for occupied spaces
    # Enemy_Ships: Array with ship name, length, 3rd index to place valid locations 
    #          ["Ship_Name", length, []]
    # Returns: x,y coordinates of chosen spot (on board)
    
    # What is a good choice for N and M in NSamples?
    ValidLocations(Board, Enemy_Ships)
    PDF,n = NSamples(2000,6000,Enemy_Ships)
    
    arr = np.where(PDF==max(PDF.max()))
    x = (arr[0]+1)[0]
    y = (arr[1]+1)[0]
    #because max PDF[x][y] corresponds to Board[x+1][y+1]
        
    return x,y

In [20]:
# np.max(PDF.max())

In [21]:
ShipSearch(Board,Enemy_Ships)
# Board[x][y] = 1

(7, 7)

In [22]:
# print(x)
# print(y)
# #print(Enemy_Ships)

In [23]:
def isCollision(Field, x, y, orientation, length):
    # isCollision = [Field, x, y, orientation, length]
    # Field = holds grid to place ships on
    # x , y = desired position of ship
    # orientation = orientation of ship (0 = hoz, 1 = vert)
    # length = length of ship
    # Returns 1 if collision occurs, 0 if ship can be placed.
    
    # Out of bounds
    if (x < 0 or y < 0 or x > 9 or y > 9):
        return 1
    
    for i in range(length):
        # Horizontal Ship
        if (orientation == 0):
            if y+length > 9:
                return 1
            if Field[x][y+i]:
                return 1
        #Vertical Ship
        elif (orientation == 1):
            if x+length > 9:
                return 1
            if Field[x+i][y]:
                return 1
    
    return 0

In [24]:
def placeShip(Field, x, y, orientation, length):
    # placeShip = [Field, length, orientation, x, y]
    # Field: dictionary for occupied spaces
    # x , y = desired position of ship
    # orientation = orientation of ship (0 = hoz, 1 = vert)
    # length: length of ship to be placed
    # Returns: 1 if ship was placed, 0 otherwise
    #          and updates Field
    
    # Note: the initial point of the ship is at (x,y) and the rest of
    #         the ship is at (x+length,y) for vertical placement and
    #         (x,y+length) for horizontal placement
    
    if isCollision(Field, x, y, orientation, length):
        return 0
    else:
        for i in range(length):   
            # Vertical Ship
            if (orientation == 1):
                Field[x+i][y] = 1
            #Horizontal Ship
            elif (orientation == 0):
                Field[x][y+i] = 1
        return 1

In [25]:
def initShips():
    Ships = np.zeros((10,10))
    s = [5, 4, 3, 3, 2]
    
    for i in range(len(s)):
        placed = 0
        while not placed:
            x = random.randint(0,9)
            y = random.randint(0,9)
            o = random.randint(0,1)
            placed = placeShip(Ships,x,y,o,s[i])
    
    return Ships

In [26]:
def initEnemyShips():
    Enemy_ = np.zeros((10,10))
    s = [5, 4, 3, 3, 2]
    n = ["Carrier","Battleship","Cruiser","Submarine","Destroyer"]
    print("Place your ships!\n")
    print("""
    length: length of ship to be placed.
    orientation = orientation of ship (0 = hoz, 1 = vert).
    x , y = desired position of ship
    Note: the initial point of the ship is at (x,y) and the rest
          of the ship will go vertically downwards or horizontally
          to the right.\n
              """)
    
    for i in range(len(s)):
        placed = 0
        clear_output()
        print(Enemies)
        print("\n")
        while not placed:
            x = random.randint(0,9)
            y = random.randint(0,9)
            o = random.randint(0,1)
            placed = placeShip(Ships,x,y,o,s[i])
            print("Your " + n[i]  + " of length " + str(s[i])
                  + " has not or could not be placed.\n")
            xStr = input("Please input the desired x coordinate\n")
            x = int(xStr)-1
            while not (x >= 0 and x <= 9):
                xStr = input("That is an invalid x coordinate\n")
                x = int(xStr)
            yStr = input("Please input the desired y coordinate\n")
            y = int(yStr)-1
            while not (y >= 0 and y <= 9):
                yStr = input("That is an invalid y coordinate\n")
                y = int(yStr)-1
            oStr = input("Please input the desired orientation\n")
            o = int(oStr)
            while not (o == 0 or o == 1):
                oStr = input("That is an invalid orientation\n")
                o = int(oStr)
            placed = placeShip(Enemies,x,y,o,s[i])
    
    return Enemies

In [27]:
def ShipSink(Board, Enemy_Ships, ShipQueue):
    # ShipSink = [Board, Enemy_Ships]
    # Board: dictionary for occupied spaces
    # Enemy_Ships: Array with ship name, length, 3rd index to place valid locations 
    #          ["Ship_Name", length, []]
    # Returns: x,y coordinates of chosen spot (on board)
    
    
    return 1

In [28]:
# print("hihihi")
# time.sleep(2)
# clear_output()
# print("helloooo")

In [29]:
# Take a turn
#     if len(ShipQueue) == 0:
#         ShipSearch(Board,Enemy_Ships)
#     else:
#         ShipSink(Board,Enemy_Ships,ShipQueue)
def AITurn(Board,Enemy_Ships,Enemies,ShipQueueX,ShipQueueY):
    x,y = ShipSearch(Board,Enemy_Ships)
    Board[y][x] = 1
    print("The AI has targeted (" + str(x) + ", " + str(y) + ").")
    if isHit(Enemies,x,y):
        ShipQueueX = np.append(ShipQueueX,x)
        ShipQueueY = np.append(ShipQueueY,y)

In [ ]:
# Player's turn
def PlayerTurn(Board,AI_Ships):
    xStr = input("Please input the x coordinate you'd like to target\n")
    x = int(xStr)
    while not (x >= 1 and x <= 10):
        xStr = input("That is an invalid x coordinate\n")
        x = int(xStr)
    yStr = input("Please input the y coordinate you'd like to target\n")
    y = int(yStr)
    while not (y >= 1 and y <= 10):
        yStr = input("That is an invalid x coordinate\n")
        y = int(yStr)
    Board[y][x] = 1
    isHit(AI_Ships,x-1,y-1)
#     if isHit(AI_Ships,x-1,y-1):
#         print('hit!')
#     else:
#         print('miss!')

In [ ]:
# Actual Game?

#INITIALIZATIONS

#Game board for AI
Board = initBoard()
# print("\nBoard")
# dispBoard(Board)

#Game board for Player
PBoard = initBoard()
# print("\nPlayer's Board")
# dispBoard(PBoard)

#Enemy ships
# Enemies = initEnemies()
# print("\nPlayer's Positions")
# print(Enemies)
#TAKE OUT FOR REAL PLAY, THIS IS JUST PLACE HOLDER.
Enemies = np.zeros((10,10))
placeShip(Enemies,0,0,0,5)
placeShip(Enemies,0,9,1,4)
placeShip(Enemies,4,4,0,3)
placeShip(Enemies,5,5,0,3)
placeShip(Enemies,8,2,0,2)
print("\nPlayer's Positions")
print(Enemies)

#Computer's ships
Ships = initShips()
print("\nAI's Positions")
print(Ships)

#Ship queue
ShipQueueX = []
ShipQueueY = []

#Whose turn toggles between 0 and 1
#Turn = 0 is the AI's turn, Turn = 1 is the Player's turn
Turn = 0
end = 2

print("\n-=-=-=-=-=-=-=-=-=-=START-GAME=-=-=-=-=-=-=-=-=-=-\n")

GameOver = 0
#START GAME
while ~GameOver and end > 0:
    if Turn:
        PlayerTurn(PBoard,Ships)
        print("\nPlayer's Playing Board")
        dispBoard(PBoard)
    else:
        AITurn(Board,Enemy_Ships,Enemies,ShipQueueX,ShipQueueY)
        # clear_output()
        print("\nAI's Playing Board")
        dispBoard(Board)
        # print("\nShipQueues")
        # print(ShipQueueX)
        # print(ShipQueueY)
    print("\n-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")
    Turn = 1-Turn
    end -= 1

# while ~GameOver
#     #if Player's turn:
#         #take input and run on Computer's boards
#     #otherwise is Computer's turn:
#         #take a turn



Player's Positions
[[ 1.  1.  1.  1.  1.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.  1.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  1.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]

AI's Positions
[[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  1.  0.  0.  1.  1.  1.  1.  0.]
 [ 0.  0.  1.  0.  0.  1.  0.  0.  1.  0.]
 [ 1.  0.  1.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  1.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]

-=-=-=-=-=-=-=-=-=-=START-GAME=-=-=-=-=-=-=-=-=-=-

The AI has targeted (6, 5).
miss!

AI's Playing B

# To Do

1. Code should be divided into "Ship Search" and "Ship Sink" methods
    * Ship Search - tries to find ship to sink (mostly written)
    * Ship Sink - tries to sink ship when one is found (need to write)
2. Find space with highest probability to shoot into for "Ship Search"
3. Write "Ship Sink" method
    * If enemy ship is hit, iterate through list of possible ships that were hit to find most likely direction to start testing
    * Edge cases very important. Possible to hit multiple ships in trying tosink one so "ship sunk" queue very important
    